In [ ]:
# Import Libraries
import cv2
import numpy as np


# The gender model architecture
# https://drive.google.com/open?id=1W_moLzMlGiELyPxWiYQJ9KFaXroQ_NFQ
GENDER_MODEL = 'weights/deploy_gender.prototxt'
# The gender model pre-trained weights
# https://drive.google.com/open?id=1AW3WduLk1haTVAxHOkVS_BEzel1WXQHP
GENDER_PROTO = 'weights/gender_net.caffemodel'
# Each Caffe Model impose the shape of the input image also image preprocessing is required like mean
# substraction to eliminate the effect of illunination changes
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
# Represent the gender classes
GENDER_LIST = ['Male', 'Female']
# https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt
FACE_PROTO = "weights/deploy.prototxt.txt"
# https://raw.githubusercontent.com/opencv/opencv_3rdparty/dnn_samples_face_detector_20180205_fp16/res10_300x300_ssd_iter_140000_fp16.caffemodel
FACE_MODEL = "weights/res10_300x300_ssd_iter_140000_fp16.caffemodel"

# load face Caffe model
face_net = cv2.dnn.readNetFromCaffe(FACE_PROTO, FACE_MODEL)
# Load gender prediction model
gender_net = cv2.dnn.readNetFromCaffe(GENDER_MODEL, GENDER_PROTO)

# Initialize frame size
frame_width = 1280
frame_height = 720


def get_faces(frame, confidence_threshold=0.5):
    # convert the frame into a blob to be ready for NN input
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104, 177.0, 123.0))
    # set the image as input to the NN
    face_net.setInput(blob)
    # perform inference and get predictions
    output = np.squeeze(face_net.forward())
    # initialize the result list
    faces = []
    # Loop over the faces detected
    for i in range(output.shape[0]):
        confidence = output[i, 2]
        if confidence > confidence_threshold:
            box = output[i, 3:7] * \
                np.array([frame.shape[1], frame.shape[0],
                         frame.shape[1], frame.shape[0]])
            # convert to integers
            start_x, start_y, end_x, end_y = box.astype(np.int)
            # widen the box a little
            start_x, start_y, end_x, end_y = start_x - \
                10, start_y - 10, end_x + 10, end_y + 10
            start_x = 0 if start_x < 0 else start_x
            start_y = 0 if start_y < 0 else start_y
            end_x = 0 if end_x < 0 else end_x
            end_y = 0 if end_y < 0 else end_y
            # append to our list
            faces.append((start_x, start_y, end_x, end_y))
    return faces


def get_optimal_font_scale(text, width):
    """Determine the optimal font scale based on the hosting frame width"""
    for scale in reversed(range(0, 60, 1)):
        textSize = cv2.getTextSize(text, fontFace=cv2.FONT_HERSHEY_DUPLEX, fontScale=scale/10, thickness=1)
        new_width = textSize[0][0]
        if (new_width <= width):
            return scale/10
    return 1

# from: https://stackoverflow.com/questions/44650888/resize-an-image-without-distortion-opencv
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]
    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image
    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)
    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))
    # resize the image
    return cv2.resize(image, dim, interpolation = inter)


def predict_gender():
    """Predict the gender of the faces showing in the image"""
    # create a new cam object
    cap = cv2.VideoCapture(0)

    while True:
        _, img = cap.read()
        # resize the image, uncomment if you want to resize the image
        # img = cv2.resize(img, (frame_width, frame_height))
        # Take a copy of the initial image and resize it
        frame = img.copy()
        if frame.shape[1] > frame_width:
            frame = image_resize(frame, width=frame_width)
        # predict the faces
        faces = get_faces(frame)
        # Loop over the faces detected
        # for idx, face in enumerate(faces):
        for i, (start_x, start_y, end_x, end_y) in enumerate(faces):
            face_img = frame[start_y: end_y, start_x: end_x]
            # image --> Input image to preprocess before passing it through our dnn for classification.
            # scale factor = After performing mean substraction we can optionally scale the image by some factor. (if 1 -> no scaling)
            # size = The spatial size that the CNN expects. Options are = (224*224, 227*227 or 299*299)
            # mean = mean substraction values to be substracted from every channel of the image.
            # swapRB=OpenCV assumes images in BGR whereas the mean is supplied in RGB. To resolve this we set swapRB to True.
            blob = cv2.dnn.blobFromImage(image=face_img, scalefactor=1.0, size=(
                227, 227), mean=MODEL_MEAN_VALUES, swapRB=False, crop=False)
            # Predict Gender
            gender_net.setInput(blob)
            gender_preds = gender_net.forward()
            i = gender_preds[0].argmax()
            gender = GENDER_LIST[i]
            gender_confidence_score = gender_preds[0][i]
            # Draw the box
            label = "{}-{:.2f}%".format(gender, gender_confidence_score*100)
            print(label)
            yPos = start_y - 15
            while yPos < 15:
                yPos += 15
            # get the font scale for this image size
            optimal_font_scale = get_optimal_font_scale(label,((end_x-start_x)+25))
            box_color = (255, 0, 0) if gender == "Male" else (147, 20, 255)
            cv2.rectangle(frame, (start_x, start_y), (end_x, end_y), box_color, 2)
            # Label processed image
            cv2.putText(frame, label, (start_x, yPos),
                        cv2.FONT_HERSHEY_SIMPLEX, optimal_font_scale, box_color, 2)

            # Display processed image
        
        # frame = cv2.resize(frame, (frame_height, frame_width))
        cv2.imshow("Gender Estimator", frame)
        if cv2.waitKey(1) == ord("q"):
            break
        # uncomment if you want to save the image
        # cv2.imwrite("output.jpg", frame)
        # Cleanup
    cv2.destroyAllWindows()



if __name__ == '__main__':
    predict_gender()

Male-56.95%
Male-80.50%
Male-54.67%
Male-82.41%
Male-87.36%
Male-84.96%
Male-63.03%
Male-82.55%
Male-77.67%
Male-71.69%
Male-51.01%
Female-88.13%
Male-70.23%
Female-86.94%
Female-88.68%
Female-65.15%
Female-55.21%
Female-77.92%
Female-93.46%
Female-84.10%
Female-90.81%
Female-81.34%
Female-78.55%
Female-76.37%
Female-84.81%
Female-56.65%
Male-62.71%
Female-59.51%
Male-62.80%
Male-73.97%
Male-76.38%
Male-83.33%
Male-91.16%
Male-86.81%
Male-86.37%
Male-88.08%
Male-85.07%
Male-88.12%
Male-81.11%
Male-82.43%
Male-77.01%
Male-75.14%
Male-88.09%
Male-73.80%
Male-87.47%
Male-70.52%
Male-69.98%
Male-70.10%
Male-59.60%
Male-69.60%
Male-70.14%
Male-60.61%
Male-63.71%
Male-69.08%
Male-74.55%
Male-52.33%
Male-67.29%
Male-68.38%
Male-77.96%
Male-56.14%
Female-60.41%
Female-73.88%
Male-63.92%
Male-73.88%
Male-52.04%
Male-69.71%
Male-63.66%
Male-54.91%
Male-65.36%
Female-51.94%
Male-54.76%
Male-57.93%
Female-73.74%
Male-53.10%
Female-52.71%
Female-55.69%
Male-53.82%
Female-56.45%
Male-61.41%
Male-75.